# Counterfactual Regret Minimization

In [1]:
from time import time
from random import shuffle
from typing import List
from collections import OrderedDict
from dataclasses import dataclass

## Controle do estado

* Para a implementação do CFR é necessária a construção de um `Node` resposável por analisar o estado atual e fazer os cálculos de acordo com a estratégia atual.

* Notações importantes:
    1. $S_i$ conjunto finito de escolhas/ações do jogador $i$
    2. $A = S_1 \times \cdots \times S_n$ conjunto com todas as combinações possíveis de ações simultâneas de todos os jogadores $\therefore A = \{a_1, \cdots a_n\}$, $a$ será um perfil de ação.
    
* Logo, para um perfil $a \in A$ o 'regret' de um jogador $i$ por não ter tomado determinada ação será:
$$u(s'_i, s_{-i}) - u(a)$$

* Vetores da classe:
    1. `regret_sum` -> Soma do arrependimento de não ter tomado determinada ação no atual `info_set`
    2. `strategy` -> Probabilidades das ações que devem ser escolhidas no atual `info_set`
    3. `strategy_sum` -> Soma de todas as estratégias utilizadas no atual `info_set`

In [2]:
class Node:
    
    def __init__(self, info_set_id):
        self.info_set = info_set_id
        self.regret_sum = [0] * NUM_ACTIONS
        self.strategy = [0] * NUM_ACTIONS
        self.strategy_sum = [0] * NUM_ACTIONS
        self.n_visits = 0
    
    
    def get_strategy(self, realization_weight):
        # compute strategy through regret-matching
        
        normalizing_sum = 0
        
        for a in range(NUM_ACTIONS):
            self.strategy[a] = self.regret_sum[a] if self.regret_sum[a] > 0 else 0
            normalizing_sum += self.strategy[a]
    
        for a in range(NUM_ACTIONS):
            if normalizing_sum > 0:
                self.strategy[a] /= normalizing_sum
            else:
                self.strategy[a] = 1/NUM_ACTIONS
                
            self.strategy_sum[a] += realization_weight*self.strategy[a]
            
        return self.strategy
    
    
    def get_average_strategy(self):
        avg_strategy = [0] * NUM_ACTIONS
        normalizing_sum = sum(self.strategy_sum)
            
        for a in range(NUM_ACTIONS):
            if normalizing_sum > 0:
                avg_strategy[a] = self.strategy_sum[a]/normalizing_sum
            else:
                avg_strategy[a] = 1/NUM_ACTIONS
                
        return avg_strategy
    
    
    def __str__(self):
        # o Infomation Set é definido pela carta na mao do jogador e a sequencia de acoes
        return f'(V: {self.n_visits}) {self.info_set}: {self.get_average_strategy()}'

## Implementação CFR

* Notações importantes:
    1. $I$ é um information set
    2. $A(I)$ são as ações permitidas em $I$
    3. $\sigma^t_i$ estratégia do jogador $i$ em um InfoSet $I_i$ com ações $A(I_i)$ com as probabilidades de escolher $a \in A(I_i)$ no tempo $t$
    4. $\sigma^t$ a estratégia de todos os jogadores juntos em $t$
    5. $\sigma_{I \rightarrow a}$ perfil em que a ação $a$ sempre é escolhida em $I$
    6. $h$ sequência de ações
    7. $\pi^\sigma(h)$ probabilidade de se ter o histórico $h$ com a estratégia $\sigma$
    8. $\pi^\sigma(I)$ probabilidade de chegar no InfoSet $I$
    9. $\pi_{-i}^\sigma(I)$ probabilidade 'counterfactual' de alcançar $I$ considerando que o jogador atual $i$ tem probabilidade $1$ de se alcançar o InfoState
    10. $Z$ conjunto com todos os $h$ terminais
    11. $h \sqsubset z, \forall z \in Z$ é a notação utilizada para o prefixo de $z$, definindo um histórico necessariamente não terminal

- **Counterfactual Value** com histórico não terminal $h$:
$$v_i(\sigma, h) = \sum_{z\in Z, h \sqsubset z} \pi^\sigma_{-i}(h, z)\cdot\pi^\sigma(h, z)\cdot u_i(z)$$

Isto é, o valor de um InfoSet $I$ para o jogador $i$ utilizando a estratégia $\sigma$ com o histórico $h$ é somar a chance do adversário estar em $I$ com o histórico $h$, a chance do jogador $i$ atingir um estado terminal com a estratégia $h$ e a utility do estado terminal $z$.

- **Counterfactual Regret** de não ter escolhido $a$ em $I$:
$$ r(h, a) = v_i(\sigma_{I \rightarrow a}, h) - v_i(\sigma, h) $$
$$\therefore r(I, a) = \sum_{h \in I} r(h, a)$$

In [3]:
class KuhnPoker():
    
    def __intit__(self):
        self.payoff_value = 0
    
    def is_terminal(self, history, cards):
        n_plays = len(history)
        terminal_state = False
        
        player = n_plays % 2
        opponent = 1 - player
        
        if n_plays > 1:
            terminal_pass = (history[-1] == 'p')
            double_bet = (history[-2:] == 'bb')
            is_playercard_higher = (cards[player] > cards[opponent])

            if terminal_pass:
                if history == 'pp':
                    self.payoff_value = 1 if is_playercard_higher else -1
                else:
                    self.payoff_value = 1
                
                terminal_state = True
            elif double_bet:
                self.payoff_value = 2 if is_playercard_higher else -2
                terminal_state = True
                
        return terminal_state
    
    
    def possible_actions(self, history):
        return [0, 1]

In [4]:
def ChanceSampling_CFR(cards, history, p0, p1):
    
    n_plays = len(history)
    
    player = n_plays % 2
    opponent = 1 - player
    
    if game.is_terminal(history, cards):
        return game.payoff_value
    
    # get infoset node or create new
    info_set = f"{cards[player]}#{history}"
    node = node_map.get(info_set)
    
    if node is None:
        node = Node(info_set)
        node_map[info_set] = node
        
    node.n_visits += 1
        
    # recursively call CFR with +action +history for each action
    player_probability = (p0 if player == 0 else p1)
    strategy = node.get_strategy(player_probability)
    
    possible_actions = game.possible_actions(history)
    util = [0] * NUM_ACTIONS
    node_util = 0
    
    for action in possible_actions:
        next_history = history + string_action[action]
        
        util[action] = -ChanceSampling_CFR(cards, next_history, p0*strategy[action], p1)\
                       if player == 0 else \
                       -ChanceSampling_CFR(cards, next_history, p0, p1*strategy[action])
        
        node_util += strategy[action]*util[action]
        
    # accumulate counterfactual regret for each action
    for action in possible_actions:
        regret = util[action] - node_util
        node.regret_sum[action] += player_probability * regret
        
    return node_util

In [5]:
# GLOBAIS
string_action = {
        0: 'p',
        1: 'b'
    }
      
NUM_ACTIONS = 2
node_map = OrderedDict()
game = KuhnPoker()

def train(iterations): 
    cards = [1, 2, 3] 
    util = 0
    
    start_time = time()
    
    for i in range(iterations):
        shuffle(cards)
        util += ChanceSampling_CFR(cards, "", 1, 1)
        
    end_time = time()
    
    print(f'Elapsed time: {end_time - start_time:.2f}s')
    print(f'Avg game value: {util/iterations}')
    
    for node in node_map.values():
        print(node)

In [6]:
iterations = 1000000
train(iterations)

Elapsed time: 10.65s
Avg game value: 0.0018025749008384383
(V: 333102) 1#: [0.9999898679683701, 1.0132031629951187e-05]
(V: 333684) 2#p: [0.9999820189161003, 1.7981083899737475e-05]
(V: 333102) 1#pb: [0.9999992494715342, 7.505284658560209e-07]
(V: 333684) 2#b: [1.4984236583114563e-06, 0.9999985015763417]
(V: 332534) 3#p: [4.5108169390197695e-06, 0.999995489183061]
(V: 332534) 3#b: [1.503605646339923e-06, 0.9999984963943537]
(V: 333722) 3#: [5.331682801232757e-06, 0.9999946683171989]
(V: 333722) 3#pb: [0.1405047048759624, 0.8594952951240376]
(V: 333176) 2#: [0.9999692045906785, 3.079540932155689e-05]
(V: 333782) 1#p: [0.9999947570570012, 5.242942998723718e-06]
(V: 333176) 2#pb: [2.5276207205525507e-05, 0.9999747237927944]
(V: 333782) 1#b: [0.999998502016286, 1.4979837139210623e-06]
